In [1]:
# Ellipsoid fitting
# I. Markovsky, et al. "Consistent least squares fitting of ellipsoids." DOI: 10.1007/s00211-004-0526-9

using GLMakie
GLMakie.activate!()

In [ ]:
A = [1 0.2 0.4; -0.1 1 -0.1; 0.3 -0.2 0.95]
c = [0.3; 0.5; -0.2]
println(inv(A * A'))

σ = 0.05

N = 3
M = 300
X = Array{Float64}(undef, N, M)

using Random
rng = Xoshiro(240120)
for i = 1:M
  θ = rand(rng, Float64) * π
  ϕ = rand(rng, Float64) * 2π
  p = A * [sin(θ) * cos(ϕ); sin(θ) * sin(ϕ); cos(θ)] + c
  e = [randn(rng, Float64); randn(rng, Float64); randn(rng, Float64)] * σ
  X[:, i] = p + e
  # println((p-c)' * inv(A * A') * (p-c))
end

fig = Figure(size = (1200, 720))
ax = Axis3(fig[1, 1])
scatter!(ax, X)
display(fig)

[1.3721090533638591 -0.3909140539358579 -0.9735562825007493; -0.3909140539358579 1.2010524776232654 0.6203651813525524; -0.9735562825007492

In [ ]:
function t(k, ξ)
  if k == 0; 1
  elseif k == 1; ξ
  elseif k == 2; ξ^2 - σ^2
  elseif k == 3; ξ^3 - 3*ξ*σ^2
  elseif k == 4; ξ^4 - 6*ξ^2*σ^2 + 3*σ^4
  end
end

T = Array{Float64}(undef, 5, N, M)
for k = 0:4
  for i = 1:N
    for l = 1:M
      T[k+1, i, l] = t(k, X[i, l])
    end
  end
end
T

In [ ]:
function vec_s(A)
  v = Any[]
  N = size(A, 1)
  for j = 1:N
    for i = 1:j
      push!(v, A[i, j])
    end
  end
  println(A)
  v
end

Nβ = (N+1) * N ÷ 2 + N + 1
V1 = fill(1, N+1)
Vi = [i % (N+1) for i in 1:N+1]
MM = hcat(vec_s(Vi * V1'), vec_s(V1 * Vi'))

In [ ]:
R = zeros(Int, Nβ, Nβ, N)
for p = 1:Nβ
  for q = p:Nβ
    for i = 1:N
      R[p, q, i] = Int.(MM[p, 1] == i) + Int.(MM[p, 2] == i) +
                   Int.(MM[q, 1] == i) + Int.(MM[q, 2] == i)
    end
  end
end
R

In [ ]:
η_als = zeros(Float64, Nβ, Nβ)
for p = 1:Nβ
  for q = p:Nβ
    η_als[p, q] = sum(l -> prod(i -> T[R[p, q, i]+1, i, l], 1:N), 1:M)
  end
end
η_als

In [ ]:
D = [x for x in 1:(N*(N+1)÷2) if !isinteger(sqrt(8x + 1))]

Ψ_als = zeros(Float64, Nβ, Nβ)
for p = 1:Nβ
  for q = p:Nβ
    Ψ_als[p, q] = η_als[p, q] * (1 + Int.(p in D)) * (1 + Int.(q in D))
  end
end
for p = 2:Nβ
  for q = 1:p-1
    Ψ_als[p, q] = Ψ_als[q, p]
  end
end
Ψ_als

In [ ]:
using LinearAlgebra
λ, v = eigen(Ψ_als)
β_als = normalize(v[:, 1])

In [ ]:
function vec_s_inv(v)
  local Nβ, N, A, k
  Nβ = size(v, 1)
  N = Int.((sqrt(8Nβ + 1) - 1) / 2)
  A = Array{Float64}(undef, N, N)
  k = 1
  for j = 1:N
    for i = 1:j
      A[i, j] = v[k]
      A[j, i] = v[k]
      k += 1
    end
  end
  A
end

A_est = vec_s_inv(β_als[1 : N*(N+1)÷2])
b_est = β_als[N*(N+1)÷2+1 : Nβ-1]
d_est = β_als[Nβ]
A_est, b_est, d_est

In [ ]:
c_est = -0.5 * inv(A_est) * b_est
AA_est = A_est / (c_est' * A_est * c_est - d_est)

c_est, AA_est